English: 14649; 15244; 15379; 14884; 15207

# Setup Libraries

In [19]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993221 sha256=be9cdc0ddd99783d9beb4a11fac81ad8520ce56d3f4c4e967d57e98b0059a5c5
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [20]:
from langdetect import detect, DetectorFactory, detect_langs

DetectorFactory.seed = 0

In [21]:
pip install langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=b2c96bef3d1e54119d16616e20fc7592d28fd8211c6439f2088660629a8013c8
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [22]:
import langid

In [1]:
# -*- coding: utf-8 -*-

In [2]:
import pandas as pd

# Setup Data and Languages

In [3]:
from google.colab import drive
drive.mount('/content/drive')

folder_path = '/content/drive/My Drive/ErwModul_24/'

Mounted at /content/drive


In [4]:
path = folder_path+'results/mBlip_prompt_results.tsv'

In [5]:
df = pd.read_csv(path, sep='\t')

In [6]:
langs_decoded = {'ar': 'Arabic', 'ca': 'Catalan', 'cs': 'Czech', 'de':'German', 'es': 'Spanish', 'fr': 'French',
                 'hu': 'Hungarian', 'it': 'Italian', 'iw': 'Hebrew', 'ja': 'Japanese', 'nl': 'Dutch', 'pl': 'Polish',
                 'pt': 'Portuguese', 'ru': 'Russian', 'sv': 'Swedish', 'uk': 'Ukrainian', 'vi': 'Vietnamese', 'zh':'Chinese (Simplified)', 'zh-TW': 'Chinese (Traditional)'}

In [7]:
langs_transf = {'ar': 'ar', 'ca': 'ca', 'cs': 'cs', 'de': 'de', 'en': 'en', 'es': 'es', 'fr': 'fr', 'hu': 'hu', 'it': 'it', 'iw': 'he', 'ja': 'ja', 'nl': 'nl', 'pl': 'pl',
                'pt': 'pt', 'ru': 'ru', 'sv': 'sv', 'uk': 'uk', 'vi': 'vi', 'zh': 'zh-cn', 'zh-TW': 'zh-tw'}

In [8]:
langs_list = list(langs_decoded.values())
langs_list.append('Chinese')

# Setup Methods

In [9]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def find_en_errors(df_capt):
  errors_found = 0
  for i in range(len(df_capt)):
    gen_cap = df_capt[i]
    if not isEnglish(gen_cap):
      errors_found += 1
  return errors_found/len(df_capt)

def find_problem_context(df_context):
  return (not isEnglish(df_context) or any(l in df_context for l in langs_list))


In [10]:
assert not isEnglish('კაცი ზის სავარძელში, იყურება კამერა წინ')
assert isEnglish('Een schilderij van een paard en een man die op een paard rijden.')

In [11]:
def find_lang_errors(df_capt, lang):
  errors_found = 0
  for i in range(len(df_capt)):
    gen_cap = df_capt[i]
    l_found = []
    try:
      x = detect_langs(gen_cap)
      for l in x:
        l_found.append(l.lang)
    except:
      l_found = [langs_transf[lang]]
    if langs_transf[lang] not in l_found:
      errors_found += 1
  return errors_found

In [12]:
def find_lang_errors2(df_capt, lang):
  errors_found = 0
  for i in range(len(df_capt)):
    gen_cap = df_capt[i]
    l_found = []
    try:
      x, _ = langid.classify(gen_cap)
    except:
      continue
    if lang != 'zh-TW' and lang!= 'iw' and x != lang:
      errors_found += 1
    elif lang == 'zh-TW' and x != 'zh':
      errors_found += 1
    elif lang == 'iw' and x != 'he':
      errors_found += 1
  return errors_found

# Execution

In [13]:
df_en = df[df['lang'] == 'en'].reset_index()

In [14]:
errors_en_simple = find_en_errors(df_en['simple_caption'])
errors_en_context = find_en_errors(df_en['context_caption'])
errors_en_transl = find_en_errors(df_en['context_transl_caption'])

In [15]:
print('Simple: {}'.format(errors_en_simple))
print('Context: {}'.format(errors_en_context))
print('Transl: {}'.format(errors_en_transl))

Simple: 0.0
Context: 0.025974025974025976
Transl: 0.13766233766233765


In [24]:
errors_simple = {}
errors_context = {}
errors_transl = {}
langs = list(set(df['lang']))
for lang in langs:
  df_lang = df[df['lang'] == lang].reset_index()
  errors_simple_lang = find_lang_errors(df_lang['simple_caption'], lang)
  errors_simple_perc = round(errors_simple_lang/len(df_lang)*100, 4)
  errors_simple[lang] = (errors_simple_lang, errors_simple_perc)
  errors_context_lang = find_lang_errors(df_lang['context_caption'], lang)
  errors_context_perc = round(errors_context_lang/len(df_lang)*100, 4)
  errors_context[lang] = (errors_context_lang, errors_context_perc)
  errors_transl_lang = find_lang_errors(df_lang['context_transl_caption'], lang)
  errors_transl_perc = round(errors_transl_lang/len(df_lang)*100, 4)
  errors_transl[lang] = (errors_transl_lang, errors_transl_perc)

In [25]:
langdetects = []
for lang in langs:
  entry = {}
  entry['lang'] = lang
  entry['minimal'] = errors_simple[lang]
  entry['context'] = errors_context[lang]
  entry['transl'] = errors_transl[lang]
  langdetects.append(entry)
df_langdetect = pd.DataFrame(langdetects)
df_langdetect.to_csv(folder_path+'results/langdetect.tsv', sep="\t")
df_langdetect

,lang,minimal,context,transl
0,ca,"(363, 47.1429)","(103, 13.3766)","(44, 5.7143)"
1,hu,"(2, 0.2597)","(43, 5.5844)","(24, 3.1169)"
2,ar,"(0, 0.0)","(58, 7.5325)","(15, 1.9481)"
3,de,"(1, 0.1297)","(27, 3.5019)","(34, 4.4099)"
4,fr,"(1, 0.1299)","(29, 3.7662)","(128, 16.6234)"
5,ja,"(10, 1.297)","(371, 48.1193)","(128, 16.6018)"
6,ru,"(19, 2.4675)","(240, 31.1688)","(131, 17.013)"
7,pt,"(771, 100.0)","(176, 22.8275)","(31, 4.0208)"
8,sv,"(61, 7.9118)","(78, 10.1167)","(72, 9.3385)"
9,zh,"(578, 74.9676)","(618, 80.1556)","(413, 53.5668)"


In [27]:
errors_simple = {}
errors_context = {}
errors_transl = {}
langs = list(set(df['lang']))
for lang in langs:
  df_lang = df[df['lang'] == lang].reset_index()
  errors_simple_lang = find_lang_errors2(df_lang['simple_caption'], lang)
  errors_simple_perc = round(errors_simple_lang/len(df_lang)*100, 4)
  errors_simple[lang] = (errors_simple_lang, errors_simple_perc)
  errors_context_lang = find_lang_errors2(df_lang['context_caption'], lang)
  errors_context_perc = round(errors_context_lang/len(df_lang)*100, 4)
  errors_context[lang] = (errors_context_lang, errors_context_perc)
  errors_transl_lang = find_lang_errors2(df_lang['context_transl_caption'], lang)
  errors_transl_perc = round(errors_transl_lang/len(df_lang)*100, 4)
  errors_transl[lang] = (errors_transl_lang, errors_transl_perc)

In [28]:
langids = []
for lang in langs:
  entry = {}
  entry['lang'] = lang
  entry['minimal'] = errors_simple[lang]
  entry['context'] = errors_context[lang]
  entry['transl'] = errors_transl[lang]
  langids.append(entry)
df_langids = pd.DataFrame(langids)
df_langids.to_csv(folder_path+'results/langid.tsv', sep="\t")
df_langids

,lang,minimal,context,transl
0,ca,"(389, 50.5195)","(184, 23.8961)","(71, 9.2208)"
1,hu,"(2, 0.2597)","(73, 9.4805)","(46, 5.974)"
2,ar,"(6, 0.7792)","(75, 9.7403)","(52, 6.7532)"
3,de,"(1, 0.1297)","(45, 5.8366)","(62, 8.0415)"
4,fr,"(2, 0.2597)","(35, 4.5455)","(154, 20.0)"
5,ja,"(4, 0.5188)","(373, 48.3787)","(110, 14.2672)"
6,ru,"(88, 11.4286)","(290, 37.6623)","(175, 22.7273)"
7,pt,"(771, 100.0)","(272, 35.2789)","(91, 11.8029)"
8,sv,"(142, 18.4176)","(155, 20.1038)","(211, 27.3671)"
9,zh,"(1, 0.1297)","(1, 0.1297)","(2, 0.2594)"


In [31]:
latex_table1 = df_langdetect.to_latex(
    index=False,
    caption="langdetect: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage",
    label="tab:ld_errors",
    position="htbp",
    column_format="|l|l|l|l|",
    escape=False
)

latex_table2 = df_langids.to_latex(
    index=False,
    caption="langid: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage",
    label="tab:langid_errors",
    position="htbp",
    column_format="|l|l|l|l|",
    escape=False
)

In [32]:
print(latex_table1)

\begin{table}[htbp]
\caption{langdetect: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage}
\label{tab:ld_errors}
\begin{tabular}{|l|l|l|l|}
\toprule
lang & minimal & context & transl \\
\midrule
ca & (363, 47.1429) & (103, 13.3766) & (44, 5.7143) \\
hu & (2, 0.2597) & (43, 5.5844) & (24, 3.1169) \\
ar & (0, 0.0) & (58, 7.5325) & (15, 1.9481) \\
de & (1, 0.1297) & (27, 3.5019) & (34, 4.4099) \\
fr & (1, 0.1299) & (29, 3.7662) & (128, 16.6234) \\
ja & (10, 1.297) & (371, 48.1193) & (128, 16.6018) \\
ru & (19, 2.4675) & (240, 31.1688) & (131, 17.013) \\
pt & (771, 100.0) & (176, 22.8275) & (31, 4.0208) \\
sv & (61, 7.9118) & (78, 10.1167) & (72, 9.3385) \\
zh & (578, 74.9676) & (618, 80.1556) & (413, 53.5668) \\
es & (2, 0.2594) & (239, 30.9987) & (53, 6.8742) \\
uk & (763, 99.0909) & (96, 12.4675) & (61, 7.9221) \\
zh-TW & (365, 48.7968) & (374, 50.0) & (353, 47.1925) \\
vi & (0, 0.0) & (4, 0.5195) & (3, 0.3896) \\
nl & (2, 0.2594) &

In [33]:
print(latex_table2)

\begin{table}[htbp]
\caption{langid: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage}
\label{tab:langid_errors}
\begin{tabular}{|l|l|l|l|}
\toprule
lang & minimal & context & transl \\
\midrule
ca & (389, 50.5195) & (184, 23.8961) & (71, 9.2208) \\
hu & (2, 0.2597) & (73, 9.4805) & (46, 5.974) \\
ar & (6, 0.7792) & (75, 9.7403) & (52, 6.7532) \\
de & (1, 0.1297) & (45, 5.8366) & (62, 8.0415) \\
fr & (2, 0.2597) & (35, 4.5455) & (154, 20.0) \\
ja & (4, 0.5188) & (373, 48.3787) & (110, 14.2672) \\
ru & (88, 11.4286) & (290, 37.6623) & (175, 22.7273) \\
pt & (771, 100.0) & (272, 35.2789) & (91, 11.8029) \\
sv & (142, 18.4176) & (155, 20.1038) & (211, 27.3671) \\
zh & (1, 0.1297) & (1, 0.1297) & (2, 0.2594) \\
es & (19, 2.4643) & (293, 38.0026) & (113, 14.6563) \\
uk & (762, 98.961) & (169, 21.9481) & (126, 16.3636) \\
zh-TW & (0, 0.0) & (1, 0.1337) & (0, 0.0) \\
vi & (0, 0.0) & (8, 1.039) & (4, 0.5195) \\
nl & (3, 0.3891) & (37, 4.79

\begin{table}[htbp]
\label{tab:ld_errors}
\begin{tabular}{|l|l|l|l|}
\hline
\textbf{lang} & \textbf{minimal} & \textbf{context} & \textbf{transl} \\
\hline
ca & (363, 47.1429) & (103, 13.3766) & (44, 5.7143) \\
hu & (2, 0.2597) & (43, 5.5844) & (24, 3.1169) \\
ar & (0, 0.0) & (58, 7.5325) & (15, 1.9481) \\
de & (1, 0.1297) & (27, 3.5019) & (34, 4.4099) \\
fr & (1, 0.1299) & (29, 3.7662) & (128, 16.6234) \\
ja & (10, 1.297) & (371, 48.1193) & (128, 16.6018) \\
ru & (19, 2.4675) & (240, 31.1688) & (131, 17.013) \\
pt & (771, 100.0) & (176, 22.8275) & (31, 4.0208) \\
sv & (61, 7.9118) & (78, 10.1167) & (72, 9.3385) \\
zh & (578, 74.9676) & (618, 80.1556) & (413, 53.5668) \\
es & (2, 0.2594) & (239, 30.9987) & (53, 6.8742) \\
uk & (763, 99.0909) & (96, 12.4675) & (61, 7.9221) \\
zh-TW & (365, 48.7968) & (374, 50.0) & (353, 47.1925) \\
vi & (0, 0.0) & (4, 0.5195) & (3, 0.3896) \\
nl & (2, 0.2594) & (53, 6.8742) & (41, 5.3178) \\
pl & (12, 1.5564) & (87, 11.284) & (46, 5.9663) \\
iw & (0, 0.0) & (1, 0.1297) & (1, 0.1297) \\
en & (2, 0.2597) & (113, 14.6753) & (157, 20.3896) \\
cs & (21, 2.7273) & (212, 27.5325) & (133, 17.2727) \\
it & (742, 96.2387) & (70, 9.0791) & (97, 12.5811) \\
\hline
\end{tabular}
\caption{langdetect: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage}
\end{table}

\begin{table}[htbp]
\label{tab:langid_errors}
\begin{tabular}{|l|l|l|l|}
\hline
\textbf{lang} & \textbf{minima}l & \textbf{context} & \textbf{transl} \\
\hline
ca & (389, 50.5195) & (184, 23.8961) & (71, 9.2208) \\
hu & (2, 0.2597) & (73, 9.4805) & (46, 5.974) \\
ar & (6, 0.7792) & (75, 9.7403) & (52, 6.7532) \\
de & (1, 0.1297) & (45, 5.8366) & (62, 8.0415) \\
fr & (2, 0.2597) & (35, 4.5455) & (154, 20.0) \\
ja & (4, 0.5188) & (373, 48.3787) & (110, 14.2672) \\
ru & (88, 11.4286) & (290, 37.6623) & (175, 22.7273) \\
pt & (771, 100.0) & (272, 35.2789) & (91, 11.8029) \\
sv & (142, 18.4176) & (155, 20.1038) & (211, 27.3671) \\
zh & (1, 0.1297) & (1, 0.1297) & (2, 0.2594) \\
es & (19, 2.4643) & (293, 38.0026) & (113, 14.6563) \\
uk & (762, 98.961) & (169, 21.9481) & (126, 16.3636) \\
zh-TW & (0, 0.0) & (1, 0.1337) & (0, 0.0) \\
vi & (0, 0.0) & (8, 1.039) & (4, 0.5195) \\
nl & (3, 0.3891) & (37, 4.799) & (39, 5.0584) \\
pl & (8, 1.0376) & (127, 16.4721) & (62, 8.0415) \\
iw & (1, 0.1297) & (2, 0.2594) & (1, 0.1297) \\
en & (3, 0.3896) & (73, 9.4805) & (156, 20.2597) \\
cs & (71, 9.2208) & (266, 34.5455) & (182, 23.6364) \\
it & (745, 96.6278) & (114, 14.786) & (162, 21.0117) \\
\hline
\end{tabular}
\caption{langid: For each of 20 languages a tuple consisting of number of incorrect language output and their percentage}
\end{table}